# Data will be grabbed from XTB

In [1]:
import xAPIConnector
from dotenv import load_dotenv
import os
import time
import json


def get_price(instrument: str):
    payload = {"period": 43200, "start": 1640998800000,
               "symbol": instrument}
    load_dotenv()
    userId = os.environ.get("LOGIN")
    password = os.environ.get("PASSWORD")
    client = xAPIConnector.APIClient()
    login_response = client.execute(xAPIConnector.loginCommand(userId=userId, password=password))
    ssid = login_response['streamSessionId']
    resp = client.commandExecute('getChartLastRequest', dict(info=payload))
    client.disconnect()
    return resp

def show_symbols():
    load_dotenv()
    userId = os.environ.get("LOGIN")
    password = os.environ.get("PASSWORD")
    client = xAPIConnector.APIClient()
    login_response = client.execute(xAPIConnector.loginCommand(userId=userId, password=password))
    ssid = login_response['streamSessionId']
    resp = client.commandExecute('getAllSymbols')
    client.disconnect()
    return resp["returnData"]


sample = get_price("W20")

In [2]:
raw = sample["returnData"]["rateInfos"]
for x in raw:
    print(x["ctmString"])

Jan 1, 2022, 12:00:00 AM
Feb 1, 2022, 12:00:00 AM
Mar 1, 2022, 12:00:00 AM
Apr 1, 2022, 12:00:00 AM
May 1, 2022, 12:00:00 AM
Jun 1, 2022, 12:00:00 AM
Jul 1, 2022, 12:00:00 AM
Aug 1, 2022, 12:00:00 AM
Sep 1, 2022, 12:00:00 AM
Oct 1, 2022, 12:00:00 AM
Nov 1, 2022, 12:00:00 AM
Dec 1, 2022, 12:00:00 AM
Jan 1, 2023, 12:00:00 AM
Feb 1, 2023, 12:00:00 AM
Mar 1, 2023, 12:00:00 AM
Apr 1, 2023, 12:00:00 AM
May 1, 2023, 12:00:00 AM


In [3]:
#symbols = show_symbols()
#with open("data.json", "w") as file:
#    json_obj = json.dumps(symbols, indent=4)
#    file.write(json_obj)

import pandas as pd

data = pd.read_json("data.json")
print(data.columns)
print(data.shape)

Index(['symbol', 'currency', 'categoryName', 'currencyProfit', 'quoteId',
       'quoteIdCross', 'marginMode', 'profitMode', 'pipsPrecision',
       'contractSize', 'exemode', 'time', 'expiration', 'stopsLevel',
       'precision', 'swapType', 'stepRuleId', 'type', 'instantMaxVolume',
       'groupName', 'description', 'longOnly', 'trailingEnabled',
       'marginHedgedStrong', 'swapEnable', 'percentage', 'bid', 'ask', 'high',
       'low', 'lotMin', 'lotMax', 'lotStep', 'tickSize', 'tickValue',
       'swapLong', 'swapShort', 'leverage', 'spreadRaw', 'spreadTable',
       'starting', 'swap_rollover3days', 'marginMaintenance', 'marginHedged',
       'initialMargin', 'timeString', 'shortSelling', 'currencyPair'],
      dtype='object')
(6018, 48)


In [4]:
data = data[(data.currency == "PLN") & (data.categoryName == "STC")]
data = data['symbol']
to_pop = ["POZ.PL"]

print(data.shape)
data = list(data)
for x in to_pop:
    data.remove(x)

(455,)


In [5]:
import numpy as np
def stock_calc(data):
    price = []
    for idx in range(len(data)-1):
        tmp = ((data[idx+1]["open"]+data[idx+1]["close"]) - (data[idx]["open"]+data[idx]["close"])) / (data[idx]["open"]+data[idx]["close"])
        price.append(tmp)
    price = np.array(price)
    return np.mean(price).__round__(2), np.std(price).__round__(2)

def parser(name: str):
    stock_data = get_price(name)
    return stock_calc(stock_data["returnData"]["rateInfos"])

def check_stock(name: str):
    stock_data = get_price(name)
    for x in stock_data["returnData"]["rateInfos"]:
        print(x)

In [6]:
sus = 'GRC.PL'
print(check_stock(sus))
print(parser(sus))

{'ctm': 1640991600000, 'ctmString': 'Jan 1, 2022, 12:00:00 AM', 'open': 6650.0, 'close': -500.0, 'high': 250.0, 'low': -1050.0, 'vol': 2190.0}
{'ctm': 1643670000000, 'ctmString': 'Feb 1, 2022, 12:00:00 AM', 'open': 6000.0, 'close': -50.0, 'high': 300.0, 'low': -800.0, 'vol': 1538.0}
{'ctm': 1646089200000, 'ctmString': 'Mar 1, 2022, 12:00:00 AM', 'open': 5950.0, 'close': -100.0, 'high': 100.0, 'low': -700.0, 'vol': 858.0}
{'ctm': 1648764000000, 'ctmString': 'Apr 1, 2022, 12:00:00 AM', 'open': 5900.0, 'close': 50.0, 'high': 350.0, 'low': -150.0, 'vol': 706.0}
{'ctm': 1651356000000, 'ctmString': 'May 1, 2022, 12:00:00 AM', 'open': 6100.0, 'close': 800.0, 'high': 1750.0, 'low': -100.0, 'vol': 1107.0}
{'ctm': 1654034400000, 'ctmString': 'Jun 1, 2022, 12:00:00 AM', 'open': 6900.0, 'close': -800.0, 'high': 200.0, 'low': -1050.0, 'vol': 740.0}
{'ctm': 1656626400000, 'ctmString': 'Jul 1, 2022, 12:00:00 AM', 'open': 6150.0, 'close': -250.0, 'high': 300.0, 'low': -250.0, 'vol': 407.0}
{'ctm': 165

In [7]:
from IPython.display import clear_output

In [8]:
stocks = pd.DataFrame(columns = ["return", "risk"], index=data)
i = 0
cnt = stocks.shape[0]
for x in data:
    i += 1
    mn, std = parser(x)
    stocks.loc[x] = [mn, std]
    clear_output(wait=True)
    print(f"{i}/{cnt} Zrobione, {((i/cnt).__round__(2))*100}, spółka: {x}")

454/454 Zrobione, 100.0, spółka: ALI.PL


In [9]:
stocks.to_csv("calculated.csv")

In [10]:
print(stocks.shape[0])

454


In [11]:
stocks[stocks.isna().any(axis=1)]
stocks = stocks.dropna()
stocks[stocks.isna().any(axis=1)]

,return,risk


NameError: name 'stocks' is not defined